In [3]:
import anthropic
import os
import pandas as pd
import numpy as np
API_KEY = '' # your API key

In [4]:
Dir_project = '/work/desai-lab/xuanyang/Project/TwoWordTest/'
df_stim = pd.read_csv(os.path.join(Dir_project,'codes','Stim_2160_shuffle10.csv'))
df_stim

,Idx,promptID,idxInPrompt,Stim,shuffleID,shuffleSeed,rand
0,1,1,1,the truck police.,1,3583.0,0.000215
1,2,1,2,the cage rib.,1,3583.0,0.000287
2,3,1,3,the holiday summer.,1,3583.0,0.000488
3,4,1,4,the soap bottle.,1,3583.0,0.001347
4,5,1,5,the radio prison.,1,3583.0,0.001542
...,...,...,...,...,...,...,...
21595,2156,8,159,the goat cheese.,0,NaN,NaN
21596,2157,8,160,the sand beach.,0,NaN,NaN
21597,2158,8,161,the mouth pig.,0,NaN,NaN
21598,2159,8,162,the field flower.,0,NaN,NaN


In [13]:
model_name = 'claude-3-opus-20240229'

for tag in ['discrete','continuous']:

    Dir_output = os.path.join(Dir_project,'results',tag,model_name)
    instruct_txt = os.path.join(Dir_project,'codes',f'instruction_{tag}.txt')

    if not os.path.exists(Dir_output):
        os.makedirs(Dir_output)

    with open(instruct_txt) as f:
        instruct = f.readlines()
    instruct[0]    


    for idx_shuffle in range(0,10):
        df_shuffle = df_stim.loc[df_stim.shuffleID == idx_shuffle].reset_index(drop=True)
        for idx_prompt in df_shuffle.promptID.unique():
            output_txt = os.path.join(Dir_output,f'{model_name}_{tag}_shuffle{idx_shuffle}_prompt{idx_prompt}.txt')

            if not os.path.exists(output_txt):

                df_prompt = df_shuffle.loc[df_shuffle.promptID == idx_prompt].reset_index(drop=True)
                lines = '\n'.join(df_prompt.Stim.values)

                lines = instruct[0]+'\n\n'+lines

                client = anthropic.Anthropic(
                    api_key=API_KEY,
                )
                message = client.messages.create(
                    model=model_name,
                    max_tokens=4096,
                    temperature=0,
                    system=instruct[0],
                    messages=[{"role": "user",
                            "content": [{"type": "text",
                                         "text": lines}]}])

                output = [message.content[0].text]
                with open(output_txt, 'a') as f:
                    for line in output:
                        f.write(line)
                print(idx_shuffle,idx_prompt)
        

9 5
9 6
9 7
9 8
